In [24]:
import pickle
import numpy as np
import pandas as pd
from gensim.corpora import Dictionary
import spacy
from gensim.models.ldamulticore import LdaMulticore
from gensim.test.utils import datapath

In [25]:
import multiprocessing
multiprocessing.cpu_count()

8

In [26]:
jstor_df = pd.read_feather("../data/large_files/jstor_df_v1.feather")

In [27]:
len(jstor_df)

14103

In [28]:
jstor_df.head(5)

,creator,datePublished,docType,doi,id,identifier,isPartOf,issueNumber,keyphrase,language,...,sequence,tdmCategory,title,url,volumeNumber,wordCount,docSubType,sourceCategory,subTitle,hasPartTitle
0,[],1959-10-01,article,10.1177/001452465907100107,ark://27927/phx66812gq6,"[{'name': 'doi', 'value': '10.1177/00145246590...",Expository Times,1,"[omnipotent reigneth, god omnipotent, silence,...",[eng],...,7.0,"[Religion - Theology, Religion - Spiritual bel...",In the Study,http://doi.org/10.1177/001452465907100107,71,7382,None,None,None,None
1,[Victor Paul Furnish],2009-01-01,chapter,10.1017/CBO9780511621321.003,ark://27927/pbd6fpf5fh,"[{'name': 'isbn', 'value': '9780511621321'}, {...",Jesus According to PaulJesus According to Paul,None,"[saying, books online, jesus tradition, pauls ...",[eng],...,5.0,"[History - Historical methodology, Religion - ...",3 Sayings of Jesus in Paul's Letters,https://doi.org/10.1017/CBO9780511621321.003,None,8577,None,None,None,None
2,[Leander E. Keck],2015-01-01,chapter,None,ark://27927/phw1kd8s300,[],Christ&#39;s First Theologian,None,"[pharisaism, sandmel genius, judaism, rabbinic...",[eng],...,8.0,"[History - Historical methodology, Religion - ...",3. The Quest for Pauls Pharisaism,None,None,5354,None,None,None,None
3,[LeAnn Snow Flesher],2009-02-01,article,10.1177/003463730910600105,ark://27927/phx64fptrwj,"[{'name': 'doi', 'value': '10.1177/00346373091...",Review & Expositor: An International Baptist J...,1,"[scofield, premillennial, premillennial dispen...",[eng],...,5.0,"[Religion - Theology, Religion - Spiritual bel...",The Historical Development of Premillennial Di...,http://doi.org/10.1177/003463730910600105,106,3614,None,None,None,None
4,[A. Daunton-Fear],1995-07-01,article,10.1177/0040571X9509800404,ark://27927/phx64k1x5c2,"[{'name': 'doi', 'value': '10.1177/0040571X950...",Theology,784,"[baptism, holy spirit, communion, infant bapti...",[eng],...,4.0,[Religion - Spiritual belief systems],Resisting the Tide Christian Initiation and Co...,http://doi.org/10.1177/0040571X9509800404,98,4323,None,None,None,None


In [29]:
# in english only?
jstor_df["language"].apply(lambda x: ("eng" in x) & (len(x)==1)).sum()

14096

In [30]:
article_ids_dict = pickle.load(open("../data/article_ids_dict.pickle", "rb"))

In [44]:
dictionary = pickle.load(open("../data/dictionary_main.pickle", "rb"))
len(dictionary)

43945

In [39]:
class FriendlyCorpus:
    def __iter__(self):
        for id in ids:
            short_id_str = str(article_ids_dict[id])
            id_filepath = "../data/large_files/article_docs/{}.pickle".format(short_id_str)
            for doc in pickle.load(open(id_filepath, "rb")):
                yield doc

In [40]:
ids = jstor_df["id"].tolist()
corpus = FriendlyCorpus()  # doesn't load the corpus into memory!
print(corpus)

In [41]:
dct_corpus = Dictionary.from_corpus(corpus, id2word=dictionary)

In [57]:
dct_corpus.token2id["New_Testament"]

10574

In [59]:
dct_corpus.dfs[10574]

59456

In [42]:
vocabulary = list(dct_corpus.values())

In [45]:
len(dct_corpus)

43945

In [46]:
%%time
lda = LdaMulticore(corpus, num_topics=10, id2word=dct_corpus)

<frozen importlib._bootstrap>:228: RuntimeWarning: scipy._lib.messagestream.MessageStream size changed, may indicate binary incompatibility. Expected 56 from C header, got 64 from PyObject
<frozen importlib._bootstrap>:228: RuntimeWarning: scipy._lib.messagestream.MessageStream size changed, may indicate binary incompatibility. Expected 56 from C header, got 64 from PyObject
<frozen importlib._bootstrap>:228: RuntimeWarning: scipy._lib.messagestream.MessageStream size changed, may indicate binary incompatibility. Expected 56 from C header, got 64 from PyObject
<frozen importlib._bootstrap>:228: RuntimeWarning: scipy._lib.messagestream.MessageStream size changed, may indicate binary incompatibility. Expected 56 from C header, got 64 from PyObject
<frozen importlib._bootstrap>:228: RuntimeWarning: scipy._lib.messagestream.MessageStream size changed, may indicate binary incompatibility. Expected 56 from C header, got 64 from PyObject
<frozen importlib._bootstrap>:228: RuntimeWarning: scip

CPU times: user 3min 7s, sys: 51.3 s, total: 3min 58s
Wall time: 4min 37s


In [50]:
lda.show_topics(num_topics=10,num_words=200,log=False, formatted=True)

[(0,
  '0.133*"God" + 0.037*"life" + 0.032*"early" + 0.022*"way" + 0.018*"tradition" + 0.017*"study" + 0.016*"order" + 0.015*"death" + 0.013*"nature" + 0.011*"old" + 0.010*"der" + 0.008*"final" + 0.008*"role" + 0.008*"state" + 0.008*"Scripture" + 0.008*"literary" + 0.007*"follow" + 0.007*"period" + 0.007*"ing" + 0.007*"movement" + 0.007*"eternal" + 0.006*"age" + 0.006*"bring" + 0.006*"resurrection" + 0.006*"Son" + 0.006*"year" + 0.006*"Father" + 0.006*"revelation" + 0.006*"local" + 0.005*"Robert" + 0.005*"remain" + 0.005*"righteousness" + 0.005*"section" + 0.005*"interest" + 0.005*"white" + 0.005*"theory" + 0.005*"vision" + 0.005*"process" + 0.005*"scholarship" + 0.005*"bishop" + 0.004*"desire" + 0.004*"short" + 0.004*"system" + 0.004*"house" + 0.004*"inner" + 0.004*"outside" + 0.004*"poor" + 0.004*"Review" + 0.004*"importance" + 0.004*"suggest" + 0.004*"Origen" + 0.004*"Historical" + 0.004*"actual" + 0.004*"brother" + 0.004*"command" + 0.004*"sent" + 0.004*"speech" + 0.003*"unto" + 0.

In [55]:
lda.show_topic(0, ntops=)

[('God', 0.13299786),
 ('life', 0.03714182),
 ('early', 0.031627595),
 ('way', 0.022248834),
 ('tradition', 0.01821983),
 ('study', 0.016909594),
 ('order', 0.01596662),
 ('death', 0.015479512),
 ('nature', 0.012623976),
 ('old', 0.010890476)]

In [51]:
lda.save("../data/large_files/lda_global_v1")